In [1]:
from IPython.display import clear_output 
!pip install konlpy==0.5.2
clear_output()

In [2]:
import pandas as pd 

from konlpy import tag
from konlpy.corpus import kolaw
from konlpy.utils import pprint 

import re
import os
# from hanspell import spell_checker 

In [3]:
import platform

#운영체제에 따라 mecab설치 방법이 다름.
if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
        
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
  try:
    from konlpy.tag import Mecab
  except:
    print("please install Mecab")

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
os.getcwd()

'/content/drive/My Drive/omz/song_data'

In [32]:
# 경로만 바꾸면 됨!
path = '/content/drive/MyDrive/omz/song_data' # song data path
data_path = 'colab_data/temp_data/' # model data path
checkpoint_path = 'colab_data/temp_data/saved_models/'  # saved model path

os.chdir(path)

In [33]:
df_dance = pd.read_excel(r'./data_song_scraping/댄스_500songs.xlsx')
df_rap_hiphop = pd.read_excel(r'./data_song_scraping/랩_힙합_500songs.xlsx')
df_balad = pd.read_excel(r'./data_song_scraping/발라드_500songs.xlsx')
df_indi = pd.read_excel(r'./data_song_scraping/인디음악_500songs.xlsx')

df_fork_blues = pd.read_excel(r'./data_song_scraping/포크_블루스_500songs.xlsx')
df_r_b = pd.read_excel(r'./data_song_scraping/알앤비_소울_500songs.xlsx')
df_rock_metal = pd.read_excel(r'./data_song_scraping/록_메탈_500songs.xlsx') 
df_trot_hiphop = pd.read_excel(r'./data_song_scraping/트로트_500songs.xlsx')

In [34]:
df_allsongs = pd.concat([df_balad, df_dance, df_fork_blues, df_indi, df_r_b, df_rap_hiphop, df_rock_metal, df_trot_hiphop])

In [35]:
df_allsongs.describe()

,SongID
count,4.000000e+03
mean,2.453939e+07
std,1.392007e+07
min,3.690000e+03
25%,5.826464e+06
50%,3.218296e+07
75%,3.475607e+07
max,3.626705e+07


In [36]:
df_allsongs

,Title,SongID,Artist,Date,Genre,Lyric
0,사랑하기 싫어,36206208,지아,2023.02.24,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
1,나비무덤,36235518,포맨 (4MEN),2023.03.01,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
2,사랑인가 봐,34657844,멜로망스,2022.02.18,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
3,너의 모든 순간,4446485,성시경,2014.02.12,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
4,사랑은 늘 도망가,34061322,임영웅,2021.10.11,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
...,...,...,...,...,...,...
495,불티,36172278,최수호,2023.02.17,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
496,안동역에서,32599795,임영웅,2020.05.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
497,남행열차,83059,김수희,1989.11,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
498,킬리만자로의 표범,81123,조용필,1985.11.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."


가사 없는 곡 -> 나중에 채워야 함

In [37]:
df_allsongs[df_allsongs['Lyric'] == ' ']

,Title,SongID,Artist,Date,Genre,Lyric


In [38]:
# # 일단 빈칸 제거 
df_allsongs = df_allsongs[df_allsongs['Lyric'] != ' ']
df_allsongs.describe()

,SongID
count,4.000000e+03
mean,2.453939e+07
std,1.392007e+07
min,3.690000e+03
25%,5.826464e+06
50%,3.218296e+07
75%,3.475607e+07
max,3.626705e+07


In [39]:
df_allsongs = df_allsongs.reset_index(drop=True)
df_allsongs

,Title,SongID,Artist,Date,Genre,Lyric
0,사랑하기 싫어,36206208,지아,2023.02.24,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
1,나비무덤,36235518,포맨 (4MEN),2023.03.01,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
2,사랑인가 봐,34657844,멜로망스,2022.02.18,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
3,너의 모든 순간,4446485,성시경,2014.02.12,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
4,사랑은 늘 도망가,34061322,임영웅,2021.10.11,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
...,...,...,...,...,...,...
3995,불티,36172278,최수호,2023.02.17,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
3996,안동역에서,32599795,임영웅,2020.05.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
3997,남행열차,83059,김수희,1989.11,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
3998,킬리만자로의 표범,81123,조용필,1985.11.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."


In [40]:
df_allsongs['Lyric'].nunique() 
# df_allsongs : 장르별 df 다 합친 것
# nunique: n of unique, 데이터 중복 유무를 확인

df_allsongs.drop_duplicates(subset = ['Lyric'], inplace = True)
len(df_allsongs)

3588

In [41]:
df_allsongs = df_allsongs.reset_index(drop=True)
df_allsongs

,Title,SongID,Artist,Date,Genre,Lyric
0,사랑하기 싫어,36206208,지아,2023.02.24,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
1,나비무덤,36235518,포맨 (4MEN),2023.03.01,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
2,사랑인가 봐,34657844,멜로망스,2022.02.18,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
3,너의 모든 순간,4446485,성시경,2014.02.12,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
4,사랑은 늘 도망가,34061322,임영웅,2021.10.11,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h..."
...,...,...,...,...,...,...
3583,불티,36172278,최수호,2023.02.17,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
3584,안동역에서,32599795,임영웅,2020.05.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
3585,남행열차,83059,김수희,1989.11,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."
3586,킬리만자로의 표범,81123,조용필,1985.11.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h..."


In [42]:
df_allsongs['preprocess_Lyric_ver2'] = ''

for i in range(len(df_allsongs)): 
    lyric = df_allsongs.Lyric.iloc[i] 
    lyric = str(lyric)
    lyric = lyric.replace('<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->', "").strip() 
    lyric = lyric.replace('_x000D_\n\t\t\t\t\t\t\t', "").strip() 
    lyric = lyric.replace('\r\n\t\t\t\t\t</div>', "").strip()  
    lyric = lyric.replace('\n</div>', "").strip()
    lyric = lyric.replace('<br/>', " ").strip()
    lyric = lyric.replace('\r', "").strip()
    lyric = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", lyric).strip()
    
    df_allsongs.loc[i, 'preprocess_Lyric_ver2'] = lyric

df_allsongs.head(3)

,Title,SongID,Artist,Date,Genre,Lyric,preprocess_Lyric_ver2
0,사랑하기 싫어,36206208,지아,2023.02.24,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h...",숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 ...
1,나비무덤,36235518,포맨 (4MEN),2023.03.01,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h...",Standing beside you as sleep Wipe my tears as ...
2,사랑인가 봐,34657844,멜로망스,2022.02.18,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h...",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...


In [43]:
df_allsongs['preprocess_Lyric_ver3'] = ''


for i in range(len(df_allsongs)): 
    lyric = df_allsongs.Lyric.iloc[i] 
    
    lyric = str(lyric)
    lyric = lyric.replace('<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->', "").strip() 
    lyric = lyric.replace('_x000D_\n\t\t\t\t\t\t\t', "").strip()
    lyric = lyric.replace('\r\n\t\t\t\t\t</div>', "").strip()  
    lyric = lyric.replace('\n</div>', "").strip()
    lyric = lyric.replace('\r', "").strip()
    lyric_list = lyric.split('<br/>')
    
    #print(lyric_list)
    #print(type(lyric_list))
    
    #df_allsongs.loc[i, 'preprocess_Lyric_ver3'] = lyric_list
    
    df_allsongs.at[i, 'preprocess_Lyric_ver3'] = lyric_list
    
    #df_allsongs.iat[i, df_allsongs.columns.get_loc('preprocess_Lyric_ver3')] = lyric_list  
    
    #df_allsongs.iloc[i, 'preprocess_Lyric_ver3'] = lyric_list

In [44]:
df_allsongs

,Title,SongID,Artist,Date,Genre,Lyric,preprocess_Lyric_ver2,preprocess_Lyric_ver3
0,사랑하기 싫어,36206208,지아,2023.02.24,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h...",숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 ...,"[숨을 내쉴 때마다, 눈을 감을 때마다, 니 생각이 나 먹먹해진다, 하루 열두 시간..."
1,나비무덤,36235518,포맨 (4MEN),2023.03.01,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h...",Standing beside you as sleep Wipe my tears as ...,"[Standing beside you as sleep, Wipe my tears a..."
2,사랑인가 봐,34657844,멜로망스,2022.02.18,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h...",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,"[너와 함께 하고 싶은 일들을, 상상하는 게, 요즘 내 일상이 되고, 너의 즐거워하..."
3,너의 모든 순간,4446485,성시경,2014.02.12,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h...",이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 ...,"[이윽고 내가 한눈에, 너를 알아봤을 때, 모든 건 분명 달라지고 있었어, 내 세상..."
4,사랑은 늘 도망가,34061322,임영웅,2021.10.11,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"[눈물이 난다 이 길을 걸으면, 그 사람 손길이 자꾸 생각이 난다, 붙잡지 못하고 ..."
...,...,...,...,...,...,...,...,...
3583,불티,36172278,최수호,2023.02.17,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h...",나의 뜨거운 마음을 불같은 나의 마음을 다시 태울 수 없을까 헤어지기는 정말 싫어 ...,"[나의 뜨거운 마음을, 불같은 나의 마음을, 다시 태울 수 없을까, 헤어지기는 정말..."
3584,안동역에서,32599795,임영웅,2020.05.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h...",바람에 날려버린 허무한 맹세였나 첫눈이 내리는 날 안동역 앞에서 만나자고 약속한사람...,"[바람에 날려버린 허무한 맹세였나, 첫눈이 내리는 날 안동역 앞에서, 만나자고 약속..."
3585,남행열차,83059,김수희,1989.11,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h...",비 내리는 호남선 남행열차에 흔들리는 차창 너머로 빗물이 흐르고 내 눈물도 흐르...,"[비 내리는 호남선 남행열차에 , 흔들리는 차창 너머로 , 빗물이 흐르고 내 눈물도..."
3586,킬리만자로의 표범,81123,조용필,1985.11.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h...",먹이를 찾아 산 기슭을 어슬렁 거리는 하이에나를 본일이 있는가 짐승의 썩은 고기만...,"[먹이를 찾아, 산 기슭을 어슬렁 거리는, 하이에나를 본일이 있는가 , 짐승의 썩은..."


In [45]:
df_allsongs.to_csv('df_allsongs.csv') 

In [46]:
temp = pd.read_csv('df_allsongs.csv', lineterminator = '\n')
temp

,Unnamed: 0,Title,SongID,Artist,Date,Genre,Lyric,preprocess_Lyric_ver2,preprocess_Lyric_ver3
0,0,사랑하기 싫어,36206208,지아,2023.02.24,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h...",숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 ...,"['숨을 내쉴 때마다', '눈을 감을 때마다', '니 생각이 나 먹먹해진다', '하..."
1,1,나비무덤,36235518,포맨 (4MEN),2023.03.01,발라드,"<div class=""lyric"" id=""d_video_summary""><!-- h...",Standing beside you as sleep Wipe my tears as ...,"['Standing beside you as sleep', 'Wipe my tear..."
2,2,사랑인가 봐,34657844,멜로망스,2022.02.18,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h...",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,"['너와 함께 하고 싶은 일들을', '상상하는 게', '요즘 내 일상이 되고', '..."
3,3,너의 모든 순간,4446485,성시경,2014.02.12,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h...",이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 ...,"['이윽고 내가 한눈에', '너를 알아봤을 때', '모든 건 분명 달라지고 있었어'..."
4,4,사랑은 늘 도망가,34061322,임영웅,2021.10.11,"발라드, 국내드라마","<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."
...,...,...,...,...,...,...,...,...,...
3583,3583,불티,36172278,최수호,2023.02.17,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h...",나의 뜨거운 마음을 불같은 나의 마음을 다시 태울 수 없을까 헤어지기는 정말 싫어 ...,"['나의 뜨거운 마음을', '불같은 나의 마음을', '다시 태울 수 없을까', '헤..."
3584,3584,안동역에서,32599795,임영웅,2020.05.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h...",바람에 날려버린 허무한 맹세였나 첫눈이 내리는 날 안동역 앞에서 만나자고 약속한사람...,"['바람에 날려버린 허무한 맹세였나', '첫눈이 내리는 날 안동역 앞에서', '만나..."
3585,3585,남행열차,83059,김수희,1989.11,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h...",비 내리는 호남선 남행열차에 흔들리는 차창 너머로 빗물이 흐르고 내 눈물도 흐르...,"['비 내리는 호남선 남행열차에 ', '흔들리는 차창 너머로 ', '빗물이 흐르고 ..."
3586,3586,킬리만자로의 표범,81123,조용필,1985.11.15,성인가요/트로트,"<div class=""lyric"" id=""d_video_summary""><!-- h...",먹이를 찾아 산 기슭을 어슬렁 거리는 하이에나를 본일이 있는가 짐승의 썩은 고기만...,"['먹이를 찾아', '산 기슭을 어슬렁 거리는', '하이에나를 본일이 있는가 ', ..."


In [47]:
df_allsongs.preprocess_Lyric_ver3.iloc[12]

['내가 좋아하는 걸 더 좋아했었죠',
 '괜한 투정 부려도 화낸 적 없었죠',
 '나쁜 일이 있어도',
 '날 보면 환히 웃었던',
 '그대에게 아무것도 해 준게 없네요',
 '무언지 모르고 받기만 했던 것들',
 '나 이제야 알 것 같아요 사랑이란 걸',
 '나는 그대를 알고 사랑을 알고',
 '이별을 알았죠',
 '다시 웃게 해줄 사람도 그대뿐이에요',
 '그대 사랑이 깊어 마음이 아파',
 '눈물이 가려서',
 '한걸음도 못가잖아요',
 '차가운 내 두 손을 놓은 적 없었죠',
 '아프기라도 하면 밤새워 지키고',
 '나의 몸과 마음을 나보다 아껴주었던',
 '그대에게 못나게도 상처만 줬네요',
 '헤어지잔 말은 꺼낸 적 없었지만',
 '변해간 마음까지도 감싸주던 한사람',
 '나는 그대를 알고 사랑을 알고',
 '이별을 알았죠',
 '다시 웃게 해줄 사람도 그대뿐이에요',
 '그대 사랑이 깊어',
 '마음이 아파 눈물이 가려서',
 '한걸음도 못가서',
 '자꾸만 돌아보는 나인 걸',
 '한 달이 가고 일 년이 가고',
 '십년이 지나도',
 '그대 나를 몰라도 나를 잊어도',
 '다른 사랑해도',
 '내가 기다리는 사람은 그대뿐이에요',
 '다시 믿게 할 사람 웃게 할 사람',
 '사랑할 한사람',
 '오직 그대인 걸 아니까',
 '']